<a href="https://colab.research.google.com/github/clarissa-fernanda/colab/blob/main/Otimiza%C3%A7%C3%A3o_de_processos_Clarissa_Macedo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Metodo não estequiometrico Lagrange
import numpy as np
from scipy.optimize import root
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter
import time  # Importa o módulo de tempo

# Constants
R = 8.314 # Universal gas constant in J/(mol*K)
T = 3500 # Temperature in K
P = 51 * 1e5 # Pressure in Pa
P0 = 1e5 # Reference pressure in Pa

# Initial mole numbers
n0_N = 2
n0_O = 2
n0_H = 4



# Standard Gibbs free energy of formation (Gf/RT)
Gf_RT = np.array([-6.06608, -14.7444, -10.1769, -14.6569, -28.3447, -24.501, -38.4983, -30.9105, -28.9657, -21.4048, -26.5222, -36.6686])

# Stoichiometric coefficients for N, O, H in each species
a_N = np.array([1, 0, 0, 1, 1, 1, 0, 0, 2, 0, 0, 1])
a_O = np.array([0, 1, 0, 0, 1, 0, 1, 2, 0, 0, 1, 2])
a_H = np.array([0, 0, 1, 1, 0, 2, 2, 0, 0, 2, 1, 0])

def L(n, lambda_N, lambda_O, lambda_H):
  return Gf_RT + np.log(n / np.sum(n)) + np.log(P / P0) - (a_N * lambda_N + a_O * lambda_O + a_H * lambda_H) / (R * T)

def equations(vars):
    n = vars[:-3] # mole numbers of the 12 species
    lambda_N, lambda_O, lambda_H = vars[-3:] # Lagrange multipliers

    # Equations for minimizing Gibbs free energy
    n_final = L(n, lambda_N, lambda_O, lambda_H)

    # Atom-balance equations
    lambda_final = [np.sum(a_O * n) - n0_O, np.sum(a_H * n) - n0_H, np.sum(a_N * n) - n0_N]

    return np.append(n_final, lambda_final)

# Initial guesses
initial_guess = [0.1]*15
start_time = time.time()

solution = root(equations, initial_guess, method='lm')

# Marca o tempo final
end_time = time.time()

# Calcula o tempo total de execução
elapsed_time = end_time - start_time
# Extract results
n_final = solution.x[:-3]
print(n_final)
lambda_final = solution.x[-3:]
print(L(n_final, lambda_final[0], lambda_final[1], lambda_final[0]))

# Display results
print("Number of function evaluations:", solution.nfev)
print("Mole fractions of the 12 species:")
species = ['N', 'O', 'H', 'NH', 'NO', 'NH2', 'H2O', 'O2', 'N2', 'H2', 'OH', 'NO2']
for i, y in enumerate(n_final / np.sum(n_final)):
    print(f"y_{species[i]} = {y:.8f}")

for i, lambda_ in enumerate(lambda_final):
    print(f"lambda_{species[i]} = {(lambda_/(R*T)):.8f}")

print("Number of function evaluations:", solution.nfev)
print("Elapsed time: {:.6f} seconds".format(elapsed_time))

[5.52717340e-05 3.30647315e-02 8.07062982e-02 1.42269844e-05
 5.32679999e-02 1.28237549e-05 1.56632104e+00 7.05821030e-02
 9.73302506e-01 2.90259528e-01 2.06092692e-01 4.46654587e-05]
[-5.32907052e-15  8.88178420e-15  3.17549026e+00  3.17549026e+00
  1.77635684e-14  6.35098051e+00  6.35098051e+00 -3.55271368e-15
 -7.10542736e-15  6.35098051e+00  3.17549026e+00  0.00000000e+00]
Number of function evaluations: 330
Mole fractions of the 12 species:
y_N = 0.00001688
y_O = 0.01010004
y_H = 0.02465275
y_NH = 0.00000435
y_NO = 0.01627138
y_NH2 = 0.00000392
y_H2O = 0.47845240
y_O2 = 0.02156019
y_N2 = 0.29730745
y_H2 = 0.08866341
y_OH = 0.06295360
y_NO2 = 0.00001364
lambda_N = -13.12343143
lambda_O = -15.40779059
lambda_H = -9.94794117
Number of function evaluations: 330
Elapsed time: 0.017661 seconds


<ipython-input-15-df6237add636>:28: RuntimeWarning: invalid value encountered in log
  return Gf_RT + np.log(n / np.sum(n)) + np.log(P / P0) - (a_N * lambda_N + a_O * lambda_O + a_H * lambda_H) / (R * T)


In [ ]:
!pip install -q casadi

In [ ]:
#Metodo não estequiometrico SQP
import numpy as np
import casadi as ca
import matplotlib.pyplot as plt

# Constants
R = 8.314
T = 3500
P = 51 * 1e5
P0 = 1e5

# Initial mole numbers
n0_N = 2
n0_O = 2
n0_H = 4

# Standard Gibbs free energy of formation (Gf/RT)
Gf_RT = np.array([-6.06608, -14.7444, -10.1769, -14.6569, -28.3447, -24.501, -38.4983, -30.9105, -28.9657, -21.4048, -26.5222, -36.6686])

# Stoichiometric coefficients for N, O, H in each species
a_N = np.array([1, 0, 0, 1, 1, 1, 0, 0, 2, 0, 0, 1])
a_O = np.array([0, 1, 0, 0, 1, 0, 1, 2, 0, 0, 1, 2])
a_H = np.array([0, 0, 1, 1, 0, 2, 2, 0, 0, 2, 1, 0])

# Define CasADi optimization variables
n = ca.MX.sym('n', 12)

# Define Objective Function
f = ca.sum1(Gf_RT * n) * R * T + R * T * ca.sum1(ca.log(n / ca.sum1(n)) * n) + R * T * ca.sum1(ca.log(P / P0) * n)

# Constraints
g1 = ca.sum1(a_N * n) - n0_N
g2 = ca.sum1(a_O * n) - n0_O
g3 = ca.sum1(a_H * n) - n0_H

g = ca.vertcat(g1, g2, g3)

# Formulate NLP problem
nlp = {'x': n, 'f': f, 'g': g}

# NLP Solver options
opts = {'print_time': False,
        'print_iteration': False}

# Create NLP solver
solver = ca.nlpsol('solver', 'sqpmethod', nlp, opts)

# Initial guesses
initial_guess = [0.1] * 12

# Marca o tempo inicial
start_time = time.time()

# Solve the NLP problem
sol = solver(x0=initial_guess, lbx=[0]*12, ubx=[np.inf]*12, lbg=[0, 0, 0], ubg=[0, 0, 0])

# Marca o tempo final
end_time = time.time()

#Calcula o tempo total de execução
elapsed_time = end_time - start_time

# Extract results
n_final = sol['x'].full().flatten()
print(n_final)
# Display results
print("Number of function evaluations:", solver.stats().get('iter_count'))
print("Mole fractions of the 12 species:")
species = ['N', 'O', 'H', 'NH', 'NO', 'NH2', 'H2O', 'O2', 'N2', 'H2', 'OH', 'NO2']
for i, y in enumerate(n_final / np.sum(n_final)):
    print(f"y_{species[i]} = {y:.8f}")

print("Number of function evaluations:", solver.stats().get('iter_count'))
print("Elapsed time: {:.6f} seconds".format(elapsed_time))



qpOASES -- An Implementation of the Online Active Set Strategy.
Copyright (C) 2007-2015 by Hans Joachim Ferreau, Andreas Potschka,
Christian Kirches et al. All rights reserved.

qpOASES is distributed under the terms of the 
GNU Lesser General Public License 2.1 in the hope that it will be 
useful, but WITHOUT ANY WARRANTY; without even the implied warranty 
of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. 
See the GNU Lesser General Public License for more details.

-------------------------------------------
This is casadi::Sqpmethod.
Using exact Hessian
Number of variables:                              12
Number of constraints:                             3
Number of nonzeros in constraint Jacobian:        36
Number of nonzeros in Lagrangian Hessian:        144



####################   qpOASES  --  QP NO.   1   #####################

    Iter   |    StepLength    |       Info       |   nFX   |   nAC    
 ----------+------------------+------------------+---------+--------- 


CasADi - 2023-09-22 03:25:11 WARNING("solver:nlp_fg failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:326]
CasADi - 2023-09-22 03:25:11 WARNING("solver:nlp_fg failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:326]
CasADi - 2023-09-22 03:25:11 WARNING("solver:nlp_fg failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:326]
CasADi - 2023-09-22 03:25:11 WARNING("solver:nlp_fg failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:326]
CasADi - 2023-09-22 03:25:11 WARNING("solver:nlp_fg failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:326]


In [ ]:
#Metodo estequiometrico
import numpy as np
from scipy.optimize import least_squares
import matplotlib.pyplot as plt

# Constants
P = 51e5 # Pressure in Pa
n0_O = 2
n0_H = 4
n0_N = 2

# Equilibrium constants
K1 = 34958.797784438364033
K2 = 24130.358814470557263
K3 = 0.004887753199592
K4 = 3776.018779887437631
K5 = 7828.494883721039514
K6 = 0.041303837133328
K7 = 0.000000000001203
K8 = 0.000000000717841
K9 = 0.000000035904894


def equations(y):
    y_N, y_O, y_H, y_NH, y_NO, y_NH2, y_H2O, y_O2, y_N2, y_H2, y_OH, y_NO2, n = y

    # Calculate F_O, F_H, F_N based on n and y-values
    F_O = n0_O - (y_O + 2*y_O2 + y_NO + y_H2O + y_OH + 2*y_NO2) * n
    F_H = n0_H - (y_H + 2*y_H2 + y_NH + 2*y_H2O + y_OH + 2*y_NH2) * n
    F_N = n0_N - (y_N + 2*y_N2 + y_NH + y_NO + y_NO2 + y_NH2) * n

    eqs = np.zeros(13)

    # Equilibrium equations
    eqs[0] = K1 - (y_H**2 / y_H2) * P
    eqs[1] = K2 - (y_O**2 / y_O2) * P
    eqs[2] = K3 - (y_N**2 / y_N2) * P
    eqs[3] = K4 - ((y_H2**2 * y_O2) / y_H2O**2) * P
    eqs[4] = K5 - ((y_OH**2 * y_H2) / y_H2O**2) * P
    eqs[5] = K6 - (y_NO**2 / (y_N2 * y_O2))
    eqs[6] = K7 - (y_NO2**2 / (y_N2 * y_O**2 * P))
    eqs[7] = K8 - (y_NH**2 / (y_N2 * y_H2))
    eqs[8] = K9 - (y_NH2 / ((y_N2**0.5) * y_H2 * (P**0.5)))

    # Atom-balance equations
    eqs[9] = F_O
    eqs[10] = F_H
    eqs[11] = F_N
    eqs[12] = np.sum(y[:-1]) - 1

    return eqs

# Initial guess
initial_guess = [0.1]*13
start_time = time.time()


# Solve the equations using the Levenberg-Marquardt algorithm
solution = least_squares(equations, initial_guess, method='lm')
end_time = time.time()
elapsed_time = end_time - start_time
# Display results
print("Number of function evaluations:", solution.nfev)
print("Mole fractions of the 12 species:")
species = ['N', 'O', 'H', 'NH', 'NO', 'NH2', 'H2O', 'O2', 'N2', 'H2', 'OH', 'NO2']
for i, y in enumerate(solution.x[:-1]):
    print(f"y_{species[i]} = {y:.8f}")

print(f"n = {solution.x[-1]}")

print("Number of function evaluations:", solution.nfev)
print("Elapsed time: {:.6f} seconds".format(elapsed_time))

Number of function evaluations: 296
Mole fractions of the 12 species:
y_N = 0.00001688
y_O = 0.01010004
y_H = 0.02465275
y_NH = 0.00000435
y_NO = 0.01627138
y_NH2 = 0.00000392
y_H2O = 0.47845239
y_O2 = 0.02156019
y_N2 = 0.29730745
y_H2 = 0.08866341
y_OH = 0.06295360
y_NO2 = 0.00001364
n = 3.2737238920188814
Number of function evaluations: 296
Elapsed time: 0.030159 seconds


<ipython-input-11-477b9fbea723>:43: RuntimeWarning: invalid value encountered in double_scalars
  eqs[8] = K9 - (y_NH2 / ((y_N2**0.5) * y_H2 * (P**0.5)))
